# Import libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import vgg16, VGG16_Weights
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from PIL import Image
import os
import numpy as np

# Database creations using pytorch Dataset 

In [ ]:
class ImageAuthenticityDataset(Dataset):
    """Dataset for image quality assessment."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves an image and its labels by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple (image, labels) where:
                image (PIL.Image): The image.
                labels (torch.Tensor): Tensor containing quality and authenticity scores.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(os.getcwd(), self.data.iloc[idx, 3])  # image_path column
        image = Image.open(img_name).convert('RGB')
        authenticity = self.data.iloc[idx, 1]  # Authenticity column
        labels = torch.tensor([authenticity], dtype=torch.float)


        if self.transform:
            image = self.transform(image)

        return image, labels


# Definitions of the models

In [ ]:
class AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        vgg = vgg16(weights=VGG16_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in vgg.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        self.fc1 = vgg.classifier[:-1]  # Up to fc2 (4096 -> 128)
        
        # New regression head
        self.regression_head = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)  # Predict authenticity
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        features = self.fc1(x)
        predictions = self.regression_head(features)
        return predictions, features

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformations for the ImageNet dataset
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

annotations_file = 'Dataset/AIGCIQA2023/real_images_annotations.csv'

# Create the dataset
dataset = ImageAuthenticityDataset(csv_file=annotations_file, transform=data_transforms)

# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


# Create data loaders
BATCH_SIZE = 1 # Set to 1 for handling individual images

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Create a dictionary containing the data loaders
dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader,
    'test': test_dataloader
}

model = AuthenticityPredictor()
criterion = nn.MSELoss()  # Mean Squared Error Loss (regression)
optimizer = optim.Adam(model.regression_head.parameters(), lr=0.001)



## Compute for each image the importace scores of lastConv layer's channels

In [ ]:
def compute_obj_x_obj_feature_map_importance(model, dataloader, device, layer_name):
    """Computes the importance of each feature map in a convolution layer by measuring the change in 
    predictions when the feature map is zeroed out, calculated per object.
    
    Args:
        model (nn.Module): The trained model
        dataloader (DataLoader): Dataloader containing the images (with batch_size=1)
        device (str): Device to run computation on ('cuda' or 'cpu')
        layer_name (str): Name of the layer to analyze
        
    Returns:
        dict: A dictionary where keys are image indices and values are numpy arrays of 
              [feature_index, importance_score] pairs sorted by importance
    """
    # Check if importance scores are already computed
    if os.path.exists('Ranking_arrays/obj_x_obj_authenticity_importance_scores.npy'):
        print("Per-object importance scores already computed, loading from file")
        return np.load('Ranking_arrays/obj_x_obj_authenticity_importance_scores.npy', allow_pickle=True).item()
    
    model.eval()
    model.to(device)
    obj_x_obj_importance = {}
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
    
    # Make sure batch_size=1 in the dataloader
    if dataloader.batch_size != 1:
        print("Warning: Dataloader batch size should be 1 for per-object importance computation")
    
    # Create a directory for saving results if it doesn't exist
    os.makedirs('Ranking_arrays', exist_ok=True)
    
    # Process each image individually
    for img_idx, (inputs, labels) in enumerate(dataloader):
        print(f"Processing image {img_idx}/{len(dataloader)}")
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Get baseline prediction
        with torch.no_grad():
            baseline_outputs, _ = model(inputs)
            baseline_error = torch.abs(baseline_outputs - labels).item()
        
        # Compute importance for each feature map for this image
        importance_scores = []
        
        for i in range(layer.out_channels):
            # Create a backup of the weights and bias
            backup_weights = layer.weight[i, ...].clone()
            backup_bias = layer.bias[i].clone() if layer.bias is not None else None
            
            # Zero out the i-th output channel
            with torch.no_grad():
                layer.weight[i, ...] = 0
                if layer.bias is not None:
                    layer.bias[i] = 0
            
            # Get prediction with the pruned feature map
            with torch.no_grad():
                pruned_outputs, _ = model(inputs)
                pruned_error = torch.abs(pruned_outputs - labels).item()
            
            # Compute importance score:
            # Higher value means removing the feature increased error (feature is important)
            # Negative value means removing the feature decreased error (feature is harmful)
            importance_score = pruned_error - baseline_error
            
            importance_scores.append([i, importance_score])
            
            # Restore weights and bias
            with torch.no_grad():
                layer.weight[i, ...] = backup_weights
                if layer.bias is not None:
                    layer.bias[i] = backup_bias
        
        # Sort importance scores for this image (highest importance first)
        sorted_scores = sorted(importance_scores, key=lambda x: x[1], reverse=True)
        obj_x_obj_importance[img_idx] = np.array(sorted_scores)
        
            
    # Save final results
    np.save('Ranking_arrays/obj_x_obj_authenticity_importance_scores.npy', obj_x_obj_importance)
    
    return obj_x_obj_importance


In [ ]:

# LAYER to prune 
LAYER = 'features.28'  # Last convolutional layer (assuming it has 512 channels)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Make sure batch size is 1
BATCH_SIZE = 1


# Load the model
model = AuthenticityPredictor()
model.load_state_dict(torch.load('Models/VGG-16_real_authenticity_finetuned.pth'))
model.eval()
model.to(DEVICE)

# Compute per-object feature importance
obj_x_obj_importance = compute_obj_x_obj_feature_map_importance(model, dataloaders['test'], DEVICE, LAYER)

print(

In [ ]:
# print the first 10 images of the test set

for i in range(10):
    img, label = test_dataset[i]
    img = img.permute(1, 2, 0).cpu().numpy()  # Convert to HWC format for visualization
    img = (img * 255).astype(np.uint8)  # Scale to [0, 255]
    img = Image.fromarray(img)
    img.show(title=f"Image {i} - Authenticity: {label.item()}")

    